# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [69]:
!pip install duckdb leafmap folium matplotlib mapclassify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.2 MB/s eta 0:00:00


In [122]:
# %pip install duckdb leafmap lonboard
import duckdb
import leafmap
import pandas as pd
import geopandas as gpd

In [123]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

nyc_data.db.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/nyc_data.db.zip'

In [124]:
con = duckdb.connect('nyc_data.db')

In [125]:
con.install_extension('spatial')
con.load_extension('spatial')

In [126]:
con.sql("SHOW TABLES")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ ROUTE_6             │
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [127]:
con.sql("SELECT * from nyc_subway_stations LIMIT 5")

┌──────────┬────────┬──────────────┬─────────────────┬─────────────────┬────────────────────────────────────────┬──────────────────────────────────┬───────────┬─────────┬─────────┬───────────┬─────────┬─────────┬─────────┬─────────────────────────────────────────────┐
│ OBJECTID │   ID   │     NAME     │    ALT_NAME     │    CROSS_ST     │               LONG_NAME                │              LABEL               │  BOROUGH  │ NGHBHD  │ ROUTES  │ TRANSFERS │  COLOR  │ EXPRESS │ CLOSED  │                    geom                     │
│  double  │ double │   varchar    │     varchar     │     varchar     │                varchar                 │             varchar              │  varchar  │ varchar │ varchar │  varchar  │ varchar │ varchar │ varchar │                  geometry                   │
├──────────┼────────┼──────────────┼─────────────────┼─────────────────┼────────────────────────────────────────┼──────────────────────────────────┼───────────┼─────────┼─────────┼───────────┼─

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [128]:
con.sql("SELECT * FROM nyc_neighborhoods WHERE name = 'Little Italy'")

┌───────────┬──────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ BORONAME  │     NAME     │                                                                                                                                        geom                                                                                                                                         │
│  varchar  │   varchar    │                                                                                                                                      geometry                                                                                                                                       │
├───────────┼──────────────┼───────────────────────────────────────────────────

In [129]:
con.sql("SELECT * FROM nyc_subway_stations WHERE ST_Within(geom, ST_GeomFromText('MULTIPOLYGON (((584702.6384288241 4508472.988402647, 584753.2898389809 4508623.442748283, 584841.1228157798 4508772.112277565, 585076.2227265466 4508672.661297781, 584852.6014370956 4507903.87643929, 584445.8329161498 4508079.830368118, 584702.6384288241 4508472.988402647)))'))")
# Spring St station is in Little Italy and it is on Route 6

┌──────────┬────────┬───────────┬──────────┬───────────────┬─────────────────────────┬───────────────┬───────────┬─────────┬─────────┬───────────┬─────────┬─────────┬─────────┬─────────────────────────────────────────────┐
│ OBJECTID │   ID   │   NAME    │ ALT_NAME │   CROSS_ST    │        LONG_NAME        │     LABEL     │  BOROUGH  │ NGHBHD  │ ROUTES  │ TRANSFERS │  COLOR  │ EXPRESS │ CLOSED  │                    geom                     │
│  double  │ double │  varchar  │ varchar  │    varchar    │         varchar         │    varchar    │  varchar  │ varchar │ varchar │  varchar  │ varchar │ varchar │ varchar │                  geometry                   │
├──────────┼────────┼───────────┼──────────┼───────────────┼─────────────────────────┼───────────────┼───────────┼─────────┼─────────┼───────────┼─────────┼─────────┼─────────┼─────────────────────────────────────────────┤
│    369.0 │  107.0 │ Spring St │ NULL     │ Lafayette Ave │ Spring St (6) Manhattan │ Spring St (6) │ Manha

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [130]:
# get all the subway stations the 6-train passes by
con.sql("CREATE OR REPLACE TABLE ROUTE_6 AS SELECT ID, NAME AS STATION_NAME, LABEL AS STATION_LABEL, ROUTES, geom FROM nyc_subway_stations WHERE ROUTES LIKE '%6%'")

In [131]:
con.sql("SELECT * FROM ROUTE_6 WHERE STATION_NAME = 'Spring St'")

┌────────┬──────────────┬───────────────┬─────────┬─────────────────────────────────────────────┐
│   ID   │ STATION_NAME │ STATION_LABEL │ ROUTES  │                    geom                     │
│ double │   varchar    │    varchar    │ varchar │                  geometry                   │
├────────┼──────────────┼───────────────┼─────────┼─────────────────────────────────────────────┤
│  107.0 │ Spring St    │ Spring St (6) │ 6       │ POINT (584696.3146033001 4508413.075791954) │
└────────┴──────────────┴───────────────┴─────────┴─────────────────────────────────────────────┘

In [132]:
nyc_neighborhoods = con.sql("SELECT * FROM nyc_neighborhoods")

In [186]:
# my answer
route_6_neighborhoods = con.sql("SELECT r.STATION_NAME, STATION_LABEL, ROUTES, BORONAME, n.NAME AS NEIGHBORHOOD, ST_AsText(r.geom) AS STATION_GEOM, ST_AsText(n.geom) AS NB_GEOM FROM ROUTE_6 AS r LEFT JOIN nyc_neighborhoods AS n ON ST_Contains(n.geom, r.geom)").df()

In [187]:
route_6_neighborhoods

,STATION_NAME,STATION_LABEL,ROUTES,BORONAME,NEIGHBORHOOD,STATION_GEOM,NB_GEOM
0,3rd Ave,3rd Ave / 138th St (6),6,The Bronx,Mott Haven,POINT (590482.6870882246 4518329.6766932225),MULTIPOLYGON (((590036.9100714123 4519499.6907...
1,Brook Ave,Brook Ave (6),6,The Bronx,Mott Haven,POINT (591158.4627344842 4517957.545755097),MULTIPOLYGON (((590036.9100714123 4519499.6907...
2,Buhre Ave,Buhre Ave (6),6,The Bronx,Parkchester,POINT (598436.4578931453 4522431.797854405),MULTIPOLYGON (((595079.6738138457 4521786.8584...
3,Castle Hill Ave,Castle Hill Ave (6),6,The Bronx,Parkchester,POINT (596821.2408605267 4520985.437910634),MULTIPOLYGON (((595079.6738138457 4521786.8584...
4,Cypress Ave,Cypress Ave (6),6,The Bronx,Mott Haven,POINT (591563.7436134552 4517738.305929306),MULTIPOLYGON (((590036.9100714123 4519499.6907...
5,E 143rd St,E 143rd St / St Mary's St (6),6,The Bronx,Mott Haven,POINT (592119.0801340764 4518038.842602893),MULTIPOLYGON (((590036.9100714123 4519499.6907...
6,E 149th St,E 149th St (6),6,The Bronx,Hunts Point,POINT (592424.9624278166 4518478.104652112),MULTIPOLYGON (((593469.7751275873 4519475.3626...
7,E 177th St,E 177th St / Parkchester (6),6,The Bronx,Soundview,POINT (595889.6025778356 4520832.971463354),MULTIPOLYGON (((596146.7069077885 4520083.8520...
8,Elder Ave,Elder Ave (6),6,The Bronx,Soundview,POINT (594509.0414594966 4520332.8512714235),MULTIPOLYGON (((596146.7069077885 4520083.8520...
9,Hunts Point Ave,Hunts Point Ave (6),6,The Bronx,South Bronx,POINT (593467.7308229264 4519434.83129059),MULTIPOLYGON (((590582.3860104522 4519453.5545...


In [181]:
# textbook answer
con.sql("SELECT DISTINCT n.NAME AS NEIGHBORHOOD, BORONAME AS BOROUGH FROM ROUTE_6 AS r LEFT JOIN nyc_neighborhoods AS n ON ST_Contains(n.geom, r.geom)").df()

,NEIGHBORHOOD,BOROUGH
0,Mott Haven,The Bronx
1,Hunts Point,The Bronx
2,South Bronx,The Bronx
3,East Harlem,Manhattan
4,Upper East Side,Manhattan
5,Financial District,Manhattan
6,Little Italy,Manhattan
7,Parkchester,The Bronx
8,Soundview,The Bronx
9,Yorkville,Manhattan


In [142]:
# series of code to check if the spatial joins have been properly executed
subway_stations_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM nyc_subway_stations").df()
subway_stations_df.head()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,POINT (583521.854408956 4507077.862599085)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,POINT (583324.4866324601 4506805.373160211)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,POINT (583304.1823994748 4506069.654048115)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,POINT (590250.10594797 4518558.019924332)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,POINT (590454.7399891173 4519145.719617855)


In [143]:
subway_stations_gdf = leafmap.df_to_gdf(subway_stations_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
subway_stations_gdf.head()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geometry
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,POINT (-74.01122 40.71038)
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,POINT (-74.01359 40.70795)
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,POINT (-74.01393 40.70132)
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,POINT (-73.92992 40.81308)
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,POINT (-73.92741 40.81835)


In [153]:
subway_stations_gdf[subway_stations_gdf['LABEL'] == 'Sound View Ave / Morrison Ave (6)'].explore()

In [148]:
nyc_neighborhoods_df = con.sql("SELECT * EXCLUDE geom, ST_AsText(geom) as geometry FROM nyc_neighborhoods").df()
nyc_neighborhoods_df.head()

,BORONAME,NAME,geometry
0,Brooklyn,Bensonhurst,MULTIPOLYGON (((582771.4257198056 4495167.4273...
1,Manhattan,East Village,MULTIPOLYGON (((585508.7534890148 4509691.2672...
2,Manhattan,West Village,MULTIPOLYGON (((583263.2776595836 4509242.6260...
3,The Bronx,Throggs Neck,MULTIPOLYGON (((597640.0090688139 4520272.7199...
4,The Bronx,Wakefield-Williamsbridge,MULTIPOLYGON (((595285.2053417757 4525938.7983...


In [149]:
nyc_neighborhoods_gdf = leafmap.df_to_gdf(nyc_neighborhoods_df, src_crs="EPSG:26918", dst_crs="EPSG:4326")
nyc_neighborhoods_gdf.head()

,BORONAME,NAME,geometry
0,Brooklyn,Bensonhurst,"MULTIPOLYGON (((-74.02167 40.60318, -73.99913 ..."
1,Manhattan,East Village,"MULTIPOLYGON (((-73.98734 40.73372, -73.97184 ..."
2,Manhattan,West Village,"MULTIPOLYGON (((-74.01399 40.72991, -74.01381 ..."
3,The Bronx,Throggs Neck,"MULTIPOLYGON (((-73.84204 40.82767, -73.8419 4..."
4,The Bronx,Wakefield-Williamsbridge,"MULTIPOLYGON (((-73.8691 40.87898, -73.86831 4..."


In [155]:
nyc_neighborhoods_gdf[nyc_neighborhoods_gdf["NAME"] == 'Soundview'].explore()

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [190]:
con.sql("SELECT * EXCLUDE geom FROM nyc_census_blocks")

┌─────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────────────┐
│      BLKID      │ POPN_TOTAL │ POPN_WHITE │ POPN_BLACK │ POPN_NATIV │ POPN_ASIAN │ POPN_OTHER │   BORONAME    │
│     varchar     │   int32    │   int32    │   int32    │   int32    │   int32    │   int32    │    varchar    │
├─────────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┼───────────────┤
│ 360850009001000 │         97 │         51 │         32 │          1 │          5 │          8 │ Staten Island │
│ 360850020011000 │         66 │         52 │          2 │          0 │          7 │          5 │ Staten Island │
│ 360850040001000 │         62 │         14 │         18 │          2 │         25 │          3 │ Staten Island │
│ 360850074001000 │        137 │         92 │         12 │          0 │         13 │         20 │ Staten Island │
│ 360850096011000 │        289 │        230 │          0 │          0 │         32 │    

In [195]:
# answer
con.sql("SELECT SUM(POPN_TOTAL) FROM nyc_neighborhoods AS n LEFT JOIN nyc_census_blocks AS c ON ST_Intersects(n.geom, c.geom) WHERE NAME = 'Battery Park'")

┌─────────────────┐
│ sum(POPN_TOTAL) │
│     int128      │
├─────────────────┤
│           17153 │
└─────────────────┘

In [160]:
con.sql("SHOW TABLES")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ ROUTE_6             │
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


In [228]:
con.sql("SELECT n.NAME AS NEIGHBORHOOD, SUM(c.POPN_TOTAL) AS POP, ROUND(ST_Area(n.geom) / 1000000, 2) AS AREA_KM2, ROUND(SUM(c.POPN_TOTAL) / (ST_Area(n.geom) / 1000000), 2) AS POP_DEN_KM2 FROM nyc_census_blocks AS c LEFT JOIN nyc_neighborhoods AS n ON ST_Intersects(c.geom, n.geom) GROUP BY NEIGHBORHOOD, n.geom ORDER BY POP_DEN_KM2 DESC")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────────────────────┬────────┬──────────┬─────────────┐
│        NEIGHBORHOOD         │  POP   │ AREA_KM2 │ POP_DEN_KM2 │
│           varchar           │ int128 │  double  │   double    │
├─────────────────────────────┼────────┼──────────┼─────────────┤
│ North Sutton Area           │  22460 │     0.33 │    68435.13 │
│ East Village                │  82266 │     1.63 │    50404.48 │
│ Chinatown                   │  16209 │     0.33 │    48825.18 │
│ Carnegie Hill               │  18763 │     0.39 │    48543.73 │
│ Upper East Side             │ 203741 │      4.2 │    48524.49 │
│ Little Italy                │  12568 │     0.27 │    46769.35 │
│ Greenwich Village           │  57224 │     1.34 │    42835.92 │
│ Gramercy                    │ 104876 │     2.51 │    41812.39 │
│ Morris Heights              │  73413 │      1.8 │    40687.92 │
│ Upper West Side             │ 214761 │     5.35 │    40152.49 │
│        ·                    │    ·   │       ·  │         ·   │
│        ·

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [ ]:
%pip install duckdb duckdb-engine jupysql

In [ ]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [ ]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.